In [1]:
from pyspark.sql import SparkSession 
spark = SparkSession. \
    builder. \
    config('spark.shuffle.useOldFetchProtocol', 'true'). \
    config("spark.sql.warehouse.dir",f"/user/itv012256/warehouse"). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()


In [2]:
spark

In [3]:
data = [
    (1, "John Doe", "2023-07-23", 300.50, "NY", None, ["shoes", "socks"], "123-456-7890"),
    (2, "Jane Doe", "2023-07-22", None, "LA", "Returned", ["t-shirt"], "456-789-1234"),
    (3, "Mike Ross", "2023-07-25", 120.00, None, "Completed", None, "789-123-4567"),
    (4, "Rachel Zane", "2023-07-24", 80.75, "TX", "Completed", ["hat"], "321-654-0987"),
    (5, "John Doe", "2023-07-23", 300.50, "NY", None, ["shoes", "socks"], "123-456-7890")
]

In [4]:
columns = ["order_id", "customer_name", "order_date", "amount", "state", "status", "items", "phone"]


In [7]:
df = spark.createDataFrame(data, columns)

In [8]:
df.show()

+--------+-------------+----------+------+-----+---------+--------------+------------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|
+--------+-------------+----------+------+-----+---------+--------------+------------+
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
|       2|     Jane Doe|2023-07-22|  null|   LA| Returned|     [t-shirt]|456-789-1234|
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed|          null|789-123-4567|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|         [hat]|321-654-0987|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
+--------+-------------+----------+------+-----+---------+--------------+------------+



In [9]:
#Remove duplicate rows.

df_no_duplicates = df.dropDuplicates()

In [10]:
df_no_duplicates.show(truncate= False)

+--------+-------------+----------+------+-----+---------+--------------+------------+
|order_id|customer_name|order_date|amount|state|status   |items         |phone       |
+--------+-------------+----------+------+-----+---------+--------------+------------+
|1       |John Doe     |2023-07-23|300.5 |NY   |null     |[shoes, socks]|123-456-7890|
|2       |Jane Doe     |2023-07-22|null  |LA   |Returned |[t-shirt]     |456-789-1234|
|4       |Rachel Zane  |2023-07-24|80.75 |TX   |Completed|[hat]         |321-654-0987|
|5       |John Doe     |2023-07-23|300.5 |NY   |null     |[shoes, socks]|123-456-7890|
|3       |Mike Ross    |2023-07-25|120.0 |null |Completed|null          |789-123-4567|
+--------+-------------+----------+------+-----+---------+--------------+------------+



In [11]:
# Remove rows with null values.

df_clean = df.dropna()

In [12]:
df_clean.show()

+--------+-------------+----------+------+-----+---------+-----+------------+
|order_id|customer_name|order_date|amount|state|   status|items|       phone|
+--------+-------------+----------+------+-----+---------+-----+------------+
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|[hat]|321-654-0987|
+--------+-------------+----------+------+-----+---------+-----+------------+



In [13]:
# Fill missing values with default values.

df_filled = df.fillna({"status":"Pending" , "amount" : 1.1, "state" : "Unknow"})

In [14]:
df_filled.show()

+--------+-------------+----------+------+------+---------+--------------+------------+
|order_id|customer_name|order_date|amount| state|   status|         items|       phone|
+--------+-------------+----------+------+------+---------+--------------+------------+
|       1|     John Doe|2023-07-23| 300.5|    NY|  Pending|[shoes, socks]|123-456-7890|
|       2|     Jane Doe|2023-07-22|   1.1|    LA| Returned|     [t-shirt]|456-789-1234|
|       3|    Mike Ross|2023-07-25| 120.0|Unknow|Completed|          null|789-123-4567|
|       4|  Rachel Zane|2023-07-24| 80.75|    TX|Completed|         [hat]|321-654-0987|
|       5|     John Doe|2023-07-23| 300.5|    NY|  Pending|[shoes, socks]|123-456-7890|
+--------+-------------+----------+------+------+---------+--------------+------------+



In [15]:
df_replace =df_filled.replace("Pending", "InProgress" , subset = ["status"])

In [16]:
df_replace.show()

+--------+-------------+----------+------+------+----------+--------------+------------+
|order_id|customer_name|order_date|amount| state|    status|         items|       phone|
+--------+-------------+----------+------+------+----------+--------------+------------+
|       1|     John Doe|2023-07-23| 300.5|    NY|InProgress|[shoes, socks]|123-456-7890|
|       2|     Jane Doe|2023-07-22|   1.1|    LA|  Returned|     [t-shirt]|456-789-1234|
|       3|    Mike Ross|2023-07-25| 120.0|Unknow| Completed|          null|789-123-4567|
|       4|  Rachel Zane|2023-07-24| 80.75|    TX| Completed|         [hat]|321-654-0987|
|       5|     John Doe|2023-07-23| 300.5|    NY|InProgress|[shoes, socks]|123-456-7890|
+--------+-------------+----------+------+------+----------+--------------+------------+



In [17]:
# Change the data type of a column.

df.printSchema()

df_datatype = df.withColumn("amount" , df["amount"].cast("integer"))
#df_trimed = df.withColumn("customer_name", trim(df["customer_name"]))
#df_regexp = df.withColumn("phone", regexp_replace("phone",  "-",   " "))

df_datatype.printSchema()

root
 |-- order_id: long (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- state: string (nullable = true)
 |-- status: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- phone: string (nullable = true)

root
 |-- order_id: long (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- state: string (nullable = true)
 |-- status: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- phone: string (nullable = true)



In [18]:
# Create a new column or update an existing column.
from pyspark.sql.functions import lit

In [19]:
df_new_column = df.withColumn("discount", lit(10))

In [20]:
df_new_column.show()

+--------+-------------+----------+------+-----+---------+--------------+------------+--------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|discount|
+--------+-------------+----------+------+-----+---------+--------------+------------+--------+
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|      10|
|       2|     Jane Doe|2023-07-22|  null|   LA| Returned|     [t-shirt]|456-789-1234|      10|
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed|          null|789-123-4567|      10|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|         [hat]|321-654-0987|      10|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|      10|
+--------+-------------+----------+------+-----+---------+--------------+------------+--------+



In [21]:
# Remove a column 

df_dropped_column = df_new_column.drop("discount")

In [22]:
df_dropped_column.show()

+--------+-------------+----------+------+-----+---------+--------------+------------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|
+--------+-------------+----------+------+-----+---------+--------------+------------+
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
|       2|     Jane Doe|2023-07-22|  null|   LA| Returned|     [t-shirt]|456-789-1234|
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed|          null|789-123-4567|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|         [hat]|321-654-0987|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
+--------+-------------+----------+------+-----+---------+--------------+------------+



In [23]:
# Rename a cloumn name 

df_renamed_column = df.withColumnRenamed("phone", "Contact_no")

In [24]:
df_renamed_column.show()

+--------+-------------+----------+------+-----+---------+--------------+------------+
|order_id|customer_name|order_date|amount|state|   status|         items|  Contact_no|
+--------+-------------+----------+------+-----+---------+--------------+------------+
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
|       2|     Jane Doe|2023-07-22|  null|   LA| Returned|     [t-shirt]|456-789-1234|
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed|          null|789-123-4567|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|         [hat]|321-654-0987|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
+--------+-------------+----------+------+-----+---------+--------------+------------+



In [25]:
 # Trim white spaces from string columns.
    
from pyspark.sql.functions import trim


In [26]:
df_trimed = df.withColumn("customer_name", trim(df["customer_name"]))
#df_regexp = df.withColumn("phone", regexp_replace("phone",  "-",   " "))

In [27]:
df.show()
df_trimed.show()

+--------+-------------+----------+------+-----+---------+--------------+------------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|
+--------+-------------+----------+------+-----+---------+--------------+------------+
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
|       2|     Jane Doe|2023-07-22|  null|   LA| Returned|     [t-shirt]|456-789-1234|
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed|          null|789-123-4567|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|         [hat]|321-654-0987|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
+--------+-------------+----------+------+-----+---------+--------------+------------+

+--------+-------------+----------+------+-----+---------+--------------+------------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|
+--------+-------------+----------+------+

In [28]:
from pyspark.sql.functions import regexp_replace

In [29]:
df_regexp = df.withColumn("phone", regexp_replace("phone",  "-",   " "))

In [30]:
df_regexp.show()

+--------+-------------+----------+------+-----+---------+--------------+------------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|
+--------+-------------+----------+------+-----+---------+--------------+------------+
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123 456 7890|
|       2|     Jane Doe|2023-07-22|  null|   LA| Returned|     [t-shirt]|456 789 1234|
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed|          null|789 123 4567|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|         [hat]|321 654 0987|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123 456 7890|
+--------+-------------+----------+------+-----+---------+--------------+------------+



In [31]:
# Filter rows based on a condition.

df_filtered = df.filter(df["amount"] > 100)

In [32]:
df_filtered.show()

+--------+-------------+----------+------+-----+---------+--------------+------------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|
+--------+-------------+----------+------+-----+---------+--------------+------------+
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed|          null|789-123-4567|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
+--------+-------------+----------+------+-----+---------+--------------+------------+



In [33]:
df_filtered_where = df.where(df["amount"] < 150)

In [34]:
df_filtered_where.show()

+--------+-------------+----------+------+-----+---------+-----+------------+
|order_id|customer_name|order_date|amount|state|   status|items|       phone|
+--------+-------------+----------+------+-----+---------+-----+------------+
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed| null|789-123-4567|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|[hat]|321-654-0987|
+--------+-------------+----------+------+-----+---------+-----+------------+



In [35]:
df_distinct =df.distinct()

In [36]:
df_distinct.show()

+--------+-------------+----------+------+-----+---------+--------------+------------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|
+--------+-------------+----------+------+-----+---------+--------------+------------+
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
|       2|     Jane Doe|2023-07-22|  null|   LA| Returned|     [t-shirt]|456-789-1234|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|         [hat]|321-654-0987|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed|          null|789-123-4567|
+--------+-------------+----------+------+-----+---------+--------------+------------+



In [37]:
# Combine two DataFrames with matching column names.
#df.column or column

df_extra = spark.createDataFrame([(6, "Harvey Specter", "2023-07-26", 500.00, "NY", "InProgress", ["jacket"], "567-890-1234"), (6, "Harvey Specter", "2023-07-26", 500.00, "NY", "InProgress", ["jacket"], "567-890-1234") ], columns)


In [38]:
columns

['order_id',
 'customer_name',
 'order_date',
 'amount',
 'state',
 'status',
 'items',
 'phone']

In [39]:
df_union = df.unionByName(df_extra)

In [40]:
df_union.show()

+--------+--------------+----------+------+-----+----------+--------------+------------+
|order_id| customer_name|order_date|amount|state|    status|         items|       phone|
+--------+--------------+----------+------+-----+----------+--------------+------------+
|       1|      John Doe|2023-07-23| 300.5|   NY|      null|[shoes, socks]|123-456-7890|
|       2|      Jane Doe|2023-07-22|  null|   LA|  Returned|     [t-shirt]|456-789-1234|
|       3|     Mike Ross|2023-07-25| 120.0| null| Completed|          null|789-123-4567|
|       4|   Rachel Zane|2023-07-24| 80.75|   TX| Completed|         [hat]|321-654-0987|
|       5|      John Doe|2023-07-23| 300.5|   NY|      null|[shoes, socks]|123-456-7890|
|       6|Harvey Specter|2023-07-26| 500.0|   NY|InProgress|      [jacket]|567-890-1234|
|       6|Harvey Specter|2023-07-26| 500.0|   NY|InProgress|      [jacket]|567-890-1234|
+--------+--------------+----------+------+-----+----------+--------------+------------+



In [41]:
df_union_duplicats = df_union.dropDuplicates()

In [42]:
df_union_duplicats.show()

+--------+--------------+----------+------+-----+----------+--------------+------------+
|order_id| customer_name|order_date|amount|state|    status|         items|       phone|
+--------+--------------+----------+------+-----+----------+--------------+------------+
|       1|      John Doe|2023-07-23| 300.5|   NY|      null|[shoes, socks]|123-456-7890|
|       6|Harvey Specter|2023-07-26| 500.0|   NY|InProgress|      [jacket]|567-890-1234|
|       2|      Jane Doe|2023-07-22|  null|   LA|  Returned|     [t-shirt]|456-789-1234|
|       4|   Rachel Zane|2023-07-24| 80.75|   TX| Completed|         [hat]|321-654-0987|
|       5|      John Doe|2023-07-23| 300.5|   NY|      null|[shoes, socks]|123-456-7890|
|       3|     Mike Ross|2023-07-25| 120.0| null| Completed|          null|789-123-4567|
+--------+--------------+----------+------+-----+----------+--------------+------------+



In [43]:
customer_df = spark.createDataFrame([(1, "John Doe", 30), (2, "Jane Doe", 28)], ["id", "name", "age"])


In [44]:
df_joined = df.join(customer_df, df.customer_name == customer_df.name, "left").write.format("noop").mode("overwrite").save()

In [45]:
df_joined.show()

AttributeError: 'NoneType' object has no attribute 'show'

In [51]:
df_joined_left = df.join(customer_df, df.customer_name == customer_df.name, "left")

In [52]:
df_joined_left.show()

+--------+-------------+----------+------+-----+---------+--------------+------------+----+--------+----+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|  id|    name| age|
+--------+-------------+----------+------+-----+---------+--------------+------------+----+--------+----+
|       2|     Jane Doe|2023-07-22|  null|   LA| Returned|     [t-shirt]|456-789-1234|   2|Jane Doe|  28|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|         [hat]|321-654-0987|null|    null|null|
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|   1|John Doe|  30|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|   1|John Doe|  30|
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed|          null|789-123-4567|null|    null|null|
+--------+-------------+----------+------+-----+---------+--------------+------------+----+--------+----+



In [53]:
df.show()

+--------+-------------+----------+------+-----+---------+--------------+------------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|
+--------+-------------+----------+------+-----+---------+--------------+------------+
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
|       2|     Jane Doe|2023-07-22|  null|   LA| Returned|     [t-shirt]|456-789-1234|
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed|          null|789-123-4567|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|         [hat]|321-654-0987|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
+--------+-------------+----------+------+-----+---------+--------------+------------+



In [54]:
from pyspark.sql.functions import sum


In [55]:
df_grouped = df.groupBy("state").agg(sum("amount").alias("total_sales"))


In [56]:
df_grouped.show()

+-----+-----------+
|state|total_sales|
+-----+-----------+
|   LA|       null|
| null|      120.0|
|   NY|      601.0|
|   TX|      80.75|
+-----+-----------+



In [57]:
df_grouped_status = df_replace.groupBy("status").agg(sum("amount").alias("total_sales"))


In [58]:
df_grouped_status.show()

+----------+-----------+
|    status|total_sales|
+----------+-----------+
|  Returned|        1.1|
| Completed|     200.75|
|InProgress|      601.0|
+----------+-----------+



In [59]:
df_pivot = df.groupBy("customer_name").pivot("state").sum("amount")


In [60]:
df_pivot.show()

+-------------+-----+----+-----+-----+
|customer_name| null|  LA|   NY|   TX|
+-------------+-----+----+-----+-----+
|     Jane Doe| null|null| null| null|
|  Rachel Zane| null|null| null|80.75|
|     John Doe| null|null|601.0| null|
|    Mike Ross|120.0|null| null| null|
+-------------+-----+----+-----+-----+



In [61]:
# Explode array columns into individual rows.

from pyspark.sql.functions import explode



In [62]:
df_exploded = df.withColumn("item", explode("items"))


In [63]:
df.show()
df_exploded.show()

+--------+-------------+----------+------+-----+---------+--------------+------------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|
+--------+-------------+----------+------+-----+---------+--------------+------------+
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
|       2|     Jane Doe|2023-07-22|  null|   LA| Returned|     [t-shirt]|456-789-1234|
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed|          null|789-123-4567|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|         [hat]|321-654-0987|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
+--------+-------------+----------+------+-----+---------+--------------+------------+

+--------+-------------+----------+------+-----+---------+--------------+------------+-------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|   item|
+--------+-------------+--

In [64]:
from pyspark.sql.functions import when
from pyspark.sql.functions import split



In [78]:
#df_datatype = df.withColumn("amount" , df["amount"].cast("integer"))
#df_trimed = df.withColumn("customer_name", trim(df["customer_name"]))
#df_regexp = df.withColumn("phone", regexp_replace("phone",  "-",   " "))
#df_split = df.withColumn("first_name", split(df["customer_name"], " ")[0])
#df_substring = df.withColumn("area_code", df["phone"].substr(1, 3))
#df_date = df.withColumn("order_date", to_date(df["order_date"], "yyyy-MM-dd"))




df_condition = df.withColumn("high_value", when(df["amount"] > 200, "Yes").otherwise("No"))



In [66]:
df_condition.show()

+--------+-------------+----------+------+-----+---------+--------------+------------+----------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|high_value|
+--------+-------------+----------+------+-----+---------+--------------+------------+----------+
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|       Yes|
|       2|     Jane Doe|2023-07-22|  null|   LA| Returned|     [t-shirt]|456-789-1234|        No|
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed|          null|789-123-4567|        No|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|         [hat]|321-654-0987|        No|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|       Yes|
+--------+-------------+----------+------+-----+---------+--------------+------------+----------+



In [67]:
df_split = df.withColumn("first_name", split(df["customer_name"], " ")[0])


In [68]:
df_split.show()

+--------+-------------+----------+------+-----+---------+--------------+------------+----------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|first_name|
+--------+-------------+----------+------+-----+---------+--------------+------------+----------+
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|      John|
|       2|     Jane Doe|2023-07-22|  null|   LA| Returned|     [t-shirt]|456-789-1234|      Jane|
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed|          null|789-123-4567|      Mike|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|         [hat]|321-654-0987|    Rachel|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|      John|
+--------+-------------+----------+------+-----+---------+--------------+------------+----------+



In [73]:
# Extract a substring from a string.

df_substring = df.withColumn("area_code", df["phone"].substr(1, 3))


In [74]:
df_substring.show()

+--------+-------------+----------+------+-----+---------+--------------+------------+---------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|area_code|
+--------+-------------+----------+------+-----+---------+--------------+------------+---------+
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|      123|
|       2|     Jane Doe|2023-07-22|  null|   LA| Returned|     [t-shirt]|456-789-1234|      456|
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed|          null|789-123-4567|      789|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|         [hat]|321-654-0987|      321|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|      123|
+--------+-------------+----------+------+-----+---------+--------------+------------+---------+



In [75]:
from pyspark.sql.functions import to_date


In [76]:
df_date = df.withColumn("order_date", to_date(df["order_date"], "yyyy-MM-dd"))


In [77]:
df_date.show()

+--------+-------------+----------+------+-----+---------+--------------+------------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|
+--------+-------------+----------+------+-----+---------+--------------+------------+
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
|       2|     Jane Doe|2023-07-22|  null|   LA| Returned|     [t-shirt]|456-789-1234|
|       3|    Mike Ross|2023-07-25| 120.0| null|Completed|          null|789-123-4567|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|         [hat]|321-654-0987|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
+--------+-------------+----------+------+-----+---------+--------------+------------+



In [82]:
df_null = df.filter(df["state"].isNull())
df_not_null = df.filter(df["state"].isNotNull())
df_null.show(truncate=False)
df_not_null.show(truncate=False)

+--------+-------------+----------+------+-----+---------+-----+------------+
|order_id|customer_name|order_date|amount|state|status   |items|phone       |
+--------+-------------+----------+------+-----+---------+-----+------------+
|3       |Mike Ross    |2023-07-25|120.0 |null |Completed|null |789-123-4567|
+--------+-------------+----------+------+-----+---------+-----+------------+

+--------+-------------+----------+------+-----+---------+--------------+------------+
|order_id|customer_name|order_date|amount|state|status   |items         |phone       |
+--------+-------------+----------+------+-----+---------+--------------+------------+
|1       |John Doe     |2023-07-23|300.5 |NY   |null     |[shoes, socks]|123-456-7890|
|2       |Jane Doe     |2023-07-22|null  |LA   |Returned |[t-shirt]     |456-789-1234|
|4       |Rachel Zane  |2023-07-24|80.75 |TX   |Completed|[hat]         |321-654-0987|
|5       |John Doe     |2023-07-23|300.5 |NY   |null     |[shoes, socks]|123-456-7890|


In [83]:
df_null = df.filter(df["state"].isNull())
df_not_null = df.filter(df["state"].isNotNull())
df_null.show 
df_not_null.show 

<bound method DataFrame.show of +--------+-------------+----------+------+-----+---------+--------------+------------+
|order_id|customer_name|order_date|amount|state|   status|         items|       phone|
+--------+-------------+----------+------+-----+---------+--------------+------------+
|       1|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
|       2|     Jane Doe|2023-07-22|  null|   LA| Returned|     [t-shirt]|456-789-1234|
|       4|  Rachel Zane|2023-07-24| 80.75|   TX|Completed|         [hat]|321-654-0987|
|       5|     John Doe|2023-07-23| 300.5|   NY|     null|[shoes, socks]|123-456-7890|
+--------+-------------+----------+------+-----+---------+--------------+------------+
>

In [84]:
df_isin = df.filter(df["state"].isin("NY", "LA"))
df_isin.show(truncate=False)

+--------+-------------+----------+------+-----+--------+--------------+------------+
|order_id|customer_name|order_date|amount|state|status  |items         |phone       |
+--------+-------------+----------+------+-----+--------+--------------+------------+
|1       |John Doe     |2023-07-23|300.5 |NY   |null    |[shoes, socks]|123-456-7890|
|2       |Jane Doe     |2023-07-22|null  |LA   |Returned|[t-shirt]     |456-789-1234|
|5       |John Doe     |2023-07-23|300.5 |NY   |null    |[shoes, socks]|123-456-7890|
+--------+-------------+----------+------+-----+--------+--------------+------------+

